In [252]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch
import torchvision
import torchvision.transforms as transforms

import sys
import os
sys.path.append(os.path.dirname(os.getcwd())+"\\MySQL Database")
from CreateDB import CreateDB

from io import BytesIO
from PIL import Image

import numpy as np

from sklearn.model_selection import train_test_split

In [11]:
db = CreateDB('root', 'Liverpool11*')

In [313]:
lengths = []
indices = []
for i in range(len(img_array)):
    if img_array[i].shape == (100, 100):
        indices.append(i)

In [315]:
len(indices)

13378

In [312]:
lengths[0] == (100,100)

True

In [275]:
query = """
SELECT image
FROM images
"""

labels_query = """
SELECT HBondDonorCount
FROM properties
"""

properties_query = """
SELECT 
"""

images = db.fetch_query('pubchem_database', query)

labels = db.fetch_query('pubchem_database', labels_query)

img_list = []

for i in range(len(images)):
    
    img_single = list(Image.open(BytesIO(images[i][0])).getdata())
    
    img_list.append(img_single)
    
img_array = np.array(img_list).reshape(-1, 100, 100)

img_tensor = torch.from_numpy(img_array)

labels_array = np.array([x[0] for x in labels])

labels_tensor = torch.from_numpy(labels_array)

X_train, X_test, y_train, y_test = train_test_split(img_tensor, labels_tensor, train_size=0.8, random_state=0)

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)

test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

In [316]:
len(img_array)

13378

In [276]:
batch_size = 4

train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True, 
                                           num_workers=2)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=2)

In [317]:
class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 3, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(3, 5, 5)
        self.fc1 = nn.Linear(484, 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 1)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [318]:
net = Net()

In [319]:
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [320]:
def train_net(num_epochs, train_loader):
    
    net.train()
    
    losses_over_time = []
    
    for epoch in range(num_epochs):

        running_loss = 0.0

        for batch_i, data in enumerate(train_loader, 0):

            inputs, labels = data

            optimizer.zero_grad()

            outputs = net(inputs.float())

            loss = criterion(outputs, labels.float())

            loss.backward()

            optimizer.step()

            running_loss += loss.item()

            if batch_i % 2000 == 1999:
                print(f'[{epoch + 1}, {batch_i + 1:5d}] loss: {running_loss/2000:.3f}')
                losses_over_time.append(running_loss)
                running_loss = 0.0
    
    return losses_over_time
                
    print('Finished Training')

In [321]:
def eval(test_loader):
    
    net.eval()
    
    test_loss_total = 0
    
    for data in test_loader:
        
        test_image, test_label = data
    
        y_pred = net(test_image.float())

        test_loss = criterion(y_pred, test_label.float())
        
        test_loss_total += test_loss.item()

    print('Average test loss is {}'.format(test_loss_total/(len(test_loader) * test_loader.batch_size)))
    
    return test_loss_total/(len(test_loader) * test_loader.batch_size)

In [322]:
loss_over_time = train_net(2, train_loader)

RuntimeError: Given groups=1, weight of size [3, 1, 5, 5], expected input[1, 4, 100, 100] to have 1 channels, but got 4 channels instead

In [307]:
test_loss = eval(test_loader)

Average test loss is 1.2123913916248288
